In [1]:
import os
import warnings
from langchain._api import LangChainDeprecationWarning
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
_ = load_dotenv(find_dotenv())

In [7]:
llm = ChatGroq(model="llama3-70b-8192")

In [15]:
from typing import Optional, List
from langchain_core.pydantic_v1 import BaseModel, Field

In [4]:
class Person(BaseModel):

    name: Optional[str] = Field(default=None, description="The name of the person.")
    lastname: Optional[str] = Field(default=None, description="The last name of the person if known.")
    country: Optional[str] = Field(default=None, description="The country of the person if known.")

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [13]:
prompt = ChatPromptTemplate.from_messages(
[
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)


prompt_with_context = prompt | llm.with_structured_output(schema=Person) 

In [14]:
comment = "I absolutely love this product! It's been a game-changer for my daily routine. The quality is top-notch and the customer service is outstanding. I've recommended it to all my friends and family. - Sarah Johnson, USA"

response = prompt_with_context.invoke({"text": comment})
response

Person(name='Sarah Johnson', lastname=None, country='USA')

In [16]:
class Data(BaseModel):
    people: List[Person]

In [17]:
prompt_with_context = prompt | llm.with_structured_output(schema=Data)

text_input = """
Alice Johnson from Canada recently reviewed a book she loved. Meanwhile, Bob Smith from the USA shared his insights on the same book in a different review. Both reviews were very insightful.
"""

response = prompt_with_context.invoke({"text": text_input})
response

Data(people=[Person(name='Alice Johnson', lastname=None, country='Canada'), Person(name='Bob Smith', lastname=None, country='USA')])

# Sentiment analisys

In [18]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    political_tendency: str = Field(
        description="The political tendency of the user"
    )
    language: str = Field(description="The language the text is written in")

tagging_chain = tagging_prompt | llm.with_structured_output(schema=Classification)

In [19]:
trump_follower = "I'm confident that President Trump's leadership and track record will once again resonate with Americans. His strong stance on economic growth and national security is exactly what our country needs at this pivotal moment. We need to bring back the proven leadership that can make America great again!"

In [20]:
response = tagging_chain.invoke({"input": trump_follower})
response

Classification(sentiment='positive', political_tendency='conservative', language='English')

In [32]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["positive", "negative","neutral"], description="The sentiment of the text")
    political_tendency: str = Field(...,
        enum=["liberal", "conservative", "neutral"],
        description="The political tendency of the user"
    )
    language: str = Field(..., enum=["english", "spanish"], description="The language the text is written in.")

In [33]:
tagging_chain = tagging_prompt | llm.with_structured_output(schema=Classification)

In [34]:
response = tagging_chain.invoke({"input": trump_follower})
response

Classification(sentiment='positive', political_tendency='conservative', language='english')